In [ ]:
! pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp /content/drive/MyDrive/Kaggle_API/kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download sanikamal/horses-or-humans-dataset

Dataset URL: https://www.kaggle.com/datasets/sanikamal/horses-or-humans-dataset
License(s): other
 63% 195M/307M [00:00<00:00, 1.89GB/s]
100% 307M/307M [00:00<00:00, 840MB/s] 


In [ ]:
!unzip "/content/horses-or-humans-dataset.zip"

Archive:  /content/horses-or-humans-dataset.zip
  inflating: horse-or-human/horse-or-human/train/horses/horse01-0.png  
  inflating: horse-or-human/horse-or-human/train/horses/horse01-1.png  
  inflating: horse-or-human/horse-or-human/train/horses/horse01-2.png  
  inflating: horse-or-human/horse-or-human/train/horses/horse01-3.png  
  inflating: horse-or-human/horse-or-human/train/horses/horse01-4.png  
  inflating: horse-or-human/horse-or-human/train/horses/horse01-5.png  
  inflating: horse-or-human/horse-or-human/train/horses/horse01-6.png  
  inflating: horse-or-human/horse-or-human/train/horses/horse01-7.png  
  inflating: horse-or-human/horse-or-human/train/horses/horse01-8.png  
  inflating: horse-or-human/horse-or-human/train/horses/horse01-9.png  
  inflating: horse-or-human/horse-or-human/train/horses/horse02-0.png  
  inflating: horse-or-human/horse-or-human/train/horses/horse02-1.png  
  inflating: horse-or-human/horse-or-human/train/horses/horse02-2.png  
  inflating: hor

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import load_model
import numpy as np


In [ ]:
# Аугментаторы
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/horse-or-human/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)
validation_generator = validation_datagen.flow_from_directory(
    '/content/horse-or-human/validation',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', AUC(name='auc')]
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │    18,940,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,034,177 (72.61 MB)

 Trainable params: 19,034,177 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator
)

Epoch 1/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 47s 1s/step - accuracy: 0.5454 - auc: 0.5495 - loss: 1.1437 - val_accuracy: 0.5000 - val_auc: 0.6951 - val_loss: 1.5255
Epoch 2/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.7022 - auc: 0.7797 - loss: 0.5694 - val_accuracy: 0.5430 - val_auc: 0.9464 - val_loss: 1.4071
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.7973 - auc: 0.8877 - loss: 0.4223 - val_accuracy: 0.5430 - val_auc: 0.6445 - val_loss: 3.8725
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.8848 - auc: 0.9555 - loss: 0.2697 - val_accuracy: 0.5117 - val_auc: 0.5898 - val_loss: 6.4212
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.8787 - auc: 0.9564 - loss: 0.2700 - val_accuracy: 0.5117 - val_auc: 0.6094 - val_loss: 4.8426
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.9127 - auc: 0.9753 - loss: 0.2123 - val_accuracy: 0.5117 - val_auc: 0.5781 - val_loss: 5.1634
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - 

In [ ]:
final_val_auc = history.history['auc'][-1] if 'auc' in history.history else history.history['val_auc'][-1]
print(f"ROC-AUC: {final_val_auc:.4f}")

ROC-AUC: 0.9983


In [ ]:
model.save('horse_or_human_classifier.keras')

In [ ]:
def predict_image(image_array):
    model = load_model('horse_or_human_classifier.keras')

    resized_image = tf.image.resize(image_array, (150, 150))
    normalized_image = resized_image / 255.0

    input_image_for_model = np.expand_dims(normalized_image, axis=0)

    prediction = model.predict(input_image_for_model)[0][0]

    if prediction < 0.5:
        label = 'Horse'
        confidence = 1 - prediction
    else:
        label = 'Human'
        confidence = prediction

    return label, float(confidence)

In [ ]:
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type='numpy', label='Загрузить изображение'),
    outputs=[
        gr.Label(num_top_classes=1, label='Prediction'),
        gr.Label(num_top_classes=1, label='Confidence')
    ],
    title='Horse/Human Classifier',
)

iface.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://260edf86ac1ac94deb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
